
# Surfs Up!

<!--lint enable no-heading-punctuation-->
![surfs-up.jpeg](Images/surfs-up.jpeg)

Congratulations! You've decided to treat yourself to a long holiday vacation in Honolulu, Hawaii! To help with your trip planning, you decided to do some climate analysis on the area. Because you are such an awesome person, you have decided to share your ninja analytical skills with the community by providing a climate analysis API. The following outlines what you need to do.



## Step 1 - Data Engineering

The climate data for Hawaii is provided through two CSV files. Start by using Python and Pandas to inspect the content of these files and clean the data.

* Create a Jupyter Notebook file called `data_engineering.ipynb` and use this to complete all of your Data Engineering tasks.

* Use Pandas to read in the measurement and station CSV files as DataFrames.

* Inspect the data for NaNs and missing values. You must decide what to do with this data.

* Save your cleaned CSV files with the prefix `clean_`.

- - -


In [1]:
#1. IMPORT LIBRARIES
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')

In [2]:
#2. READ IN DATA
measurements = pd.read_csv("Resources/hawaii_measurements.csv")
stations = pd.read_csv("Resources/hawaii_stations.csv")


In [3]:
#3. DESCRIBE MEASUREMENT OBS - SEE IF THE COUNT IS THE SAME
print(measurements.describe())
measurements.head()

               prcp          tobs
count  18103.000000  19550.000000
mean       0.160644     73.097954
std        0.468746      4.523527
min        0.000000     53.000000
25%        0.000000     70.000000
50%        0.010000     73.000000
75%        0.110000     76.000000
max       11.530000     87.000000


,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [4]:
#4. DROP MISSING VALUES
clean_measurements = measurements.dropna(how = 'any')
# clean_measurements.reset_index(level = None, inplace = True)
# clean_measurements.rename(columns= {"index": "id"}, inplace = True)
print(clean_measurements.describe())
clean_measurements.head()

               prcp          tobs
count  18103.000000  18103.000000
mean       0.160644     72.994863
std        0.468746      4.512107
min        0.000000     53.000000
25%        0.000000     70.000000
50%        0.010000     73.000000
75%        0.110000     76.000000
max       11.530000     87.000000


,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [5]:
clean_measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [6]:
#5. DESCRIBE STATIONS SEE COUNTS ARE THE SAME
print(stations.describe())
stations

#no need to do anything
clean_stations = stations
# clean_stations.reset_index(level = None, inplace = True)
# clean_stations.rename(columns= {"index": "id"}, inplace = True)
clean_stations.head()

        latitude   longitude   elevation
count   9.000000    9.000000    9.000000
mean   21.393826 -157.867098   60.977778
std     0.086442    0.103873  103.465547
min    21.271600 -158.011100    0.900000
25%    21.333100 -157.975100    7.000000
50%    21.393400 -157.837400   14.600000
75%    21.451670 -157.802500   32.900000
max    21.521300 -157.711390  306.600000


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


## Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

* Use the `engine` and connection string to create a database called `hawaii.sqlite`.

* Use `declarative_base` and create ORM classes for each table.

  * You will need a class for `Measurement` and for `Station`.

  * Make sure to define your primary keys.

* Once you have your ORM classes defined, create the tables in the database using `create_all`.

- - -


In [16]:
#1. IMPORT DEPENDENCIES
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [17]:
#2. CREATE DATABASE CONNECTION
engine = create_engine('sqlite:///Resources/hawaii.sqlite')
engine.table_names()

['measurement', 'station']

In [18]:
#3. CREATE CLASSES
class Measurement(Base):
    __tablename__ = "measurement"
    index = Column(Integer, primary_key = True)
    station	= Column(String) 
    date = Column(String)
    prcp =Column(Float)
    tobs = Column(Float)

class Station(Base):   
    __tablename__ = "station"
    index = Column(Integer, primary_key = True)
    station = Column(String)
    name = Column(String) 
    latitude = Column(Float) 
    longitude = Column(Float) 
    elevation = Column(Integer)
    

In [19]:
#4. CREATE METADATA LAYER
Base.metadata.create_all(engine)

In [20]:
#5. PASS THE DATAFRAMES TO SQL
clean_measurements.to_sql('measurement', con=engine, if_exists = 'append', index = True)
clean_stations.to_sql('station', con = engine, if_exists = 'append', index = True)
# for row in measurements.iterrows():
#     print(row[1])

# conn = engine.connect()
    
#clean_measurements.to_sql("measurement", conn, if_exists = "append", index = "False")
#clean_stations.to_sql("station", conn, if_exists = "append", index = "False")